
# Generate surface currents

Load all necessary modules

In [ ]:
using CoastalCurrents
using CoastalCurrents: Altimetry

In [ ]:
using DIVAnd_HFRadar
using PhysOcean
using GeoMapping
#using OceanPlot
using NCDatasets
using Dates
using Test
using DIVAnd
using DataStructures

In [ ]:
include("common.jl")

Load the data generated by prep_altimetry notebook/script

In [ ]:
ds = NCDataset(altimetry_fname)
mdt = ds["mdt"][:]
sla = ds["slaf"][:]
lon = ds["lon"][:]
lat = ds["lat"][:]
time = ds["time"][:]
id = ds["id"][:]

In [ ]:
len = ds["len"][:]

In [ ]:
close(ds)

In [ ]:
adt = mdt + sla

slaf = NCDatasets.loadragged(ds["slaf"],:)

figure();scatter(lon,lat,10,id)
figure();
scatter(lon,lat,10,adt);
OceanPlot.set_aspect_ratio()

In [ ]:
scale = 50
scale = 5

In [ ]:
sel = @. Dates.Month(time) == 1
sel = @. Dates.DateTime(2000,1,1) <= time < Dates.DateTime(2001,1,1)

In [ ]:
#=
sel = 1:10000
len = len[sel]

In [ ]:
lon = lon[1:sum(len)]
lat = lat[1:sum(len)]
adt = adt[1:sum(len)]
time = time[1:sum(len)]

In [ ]:
(lona,lata,timea,ua,va) = Altimetry.geostrophic_velocity(lon,lat,time,adt,len)
r = 1:1:length(ua)
=#

In [ ]:
#=
clf()
scatter(lon,lat,10,adt,vmin=-0.1,vmax = 0.08,cmap="jet"); colorbar()
quiver(lona[r],lata[r],ua[r],va[r],scale=scale,lw = 0.1)
#xlim((2.5832958255391247, 5.08659455171254))
#ylim((36.03972058860369, 38.84928070740686))

In [ ]:
#OceanPlot.plot_coastline("f")
xlim((22.25885640558503, 27.75667790863112))
ylim((31.704241622526148, 37.93893180755805))

In [ ]:
OceanPlot.plotmap()
OceanPlot.set_aspect_ratio()
savefig(expanduser("~/Figures/altimetry_currents_$(timea[1]).png"))
=#

In [ ]:
robs = sqrt.(ua.^2 + va.^2)
len = 150e3

In [ ]:
# directionobs angle in degree relative to North
# see DIVAndrun_HFRadar
directionobs = atand.(ua,va)

In [ ]:
@test ua[1] ≈ robs[1]*sind(directionobs[1])
@test va[1] ≈ robs[1]*cosd(directionobs[1])

In [ ]:
mask,(pm,pn),(xi,yi) = DIVAnd.domain(bathname,bathisglobal,lonr,latr)

In [ ]:
mask = DIVAnd.floodfill(mask) .== 1

In [ ]:
hx, hy, h = DIVAnd.load_bath(bathname, bathisglobal, lonr, latr)

In [ ]:
g = 0;
x = lona
y = lata

In [ ]:
valid = isfinite.(robs)
x = lona[valid]
y = lata[valid]
robs = robs[valid]
directionobs = directionobs[valid]

In [ ]:
epsilon2 = 2.
eps2_boundary_constraint = -1
eps2_div_constraint = -1
#eps2_boundary_constraint = 1e-9
eps2_div_constraint = 1e+1
eps2_div_constraint = 1

In [ ]:
#figure()
uri,vri,ηi = DIVAndrun_HFRadar(
    mask,h,(pm,pn),(xi,yi),(x,y),robs,directionobs,len,epsilon2;
    eps2_boundary_constraint = eps2_boundary_constraint,
    eps2_div_constraint = eps2_div_constraint,
    # eps2_Coriolis_constraint = -1,
    # f = 0.001,
    # residual = residual,
    # g = g,
    # ratio = 100,
    # lenη = (000.0, 000.0, 24 * 60 * 60. * 10),
    # maxit = 100000,
    # tol = 1e-6,
)

In [ ]:
isfile(result_filename) && rm(result_filename)

In [ ]:
# TODO
timei = first(time)
timei = [DateTime(2013,3,1)]

In [ ]:
ds = NCDataset(result_filename,"c")

In [ ]:
defVar(ds,"lon",xi[:,1],("lon",),attrib = OrderedDict(
    "long_name" => "longitude",
    "units" => "degrees_east",
    "standard_name" => "longitude"))

In [ ]:
defVar(ds,"lat",yi[1,:],("lat",),attrib = OrderedDict(
    "long_name" => "latitude",
    "units" => "degrees_north",
    "standard_name" => "latitude"))

In [ ]:
defVar(ds,"time",timei,("time",),
       attrib = OrderedDict(
           "standard_name" => "time",
           "units" => "days since 1950-01-01 00:00:00",
       ))

In [ ]:
defVar(ds,"u",uri[:,:,1:1],("lon","lat","time"),attrib = OrderedDict(
    "long_name" => "zonal velocity",
    "units" => "m/s",
    "standard_name" => "eastward_sea_water_velocity"))

In [ ]:
defVar(ds,"v",vri[:,:,1:1],("lon","lat","time"),attrib = OrderedDict(
    "long_name" => "meridional velocity",
    "units" => "m/s",
    "standard_name" => "northward_sea_water_velocity"))

In [ ]:
ds.attrib["Conventions"] = "CF-1.10"

In [ ]:
close(ds)

In [ ]:
color = sqrt.(uri.^2 + vri.^2)

In [ ]:
using PyPlot
clf()
r = CartesianIndices(( 1:2:size(mask,1) ,1:2:size(mask,2)))
r = CartesianIndices(( 1:1:size(mask,1) ,1:1:size(mask,2)))
quiver(xi[r],yi[r],uri[r],vri[r],color[r],cmap="jet")
colorbar(orientation="horizontal")
CoastalCurrents.Plotting.plotmap(bathname)
title("surface current " * join(Dates.format.((minimum(timea),maximum(timea)),"yyyy-mm-dd")," - "))
savefig(expanduser("~/Figures/altimetry_currents_DIVAnd.png"),dpi=300)

=

In [ ]:
clf()
r = CartesianIndices(( 1:1:size(mask,1) ,1:1:size(mask,2)))
quiver(xi[r],yi[r],uri[r],vri[r],color[r],cmap="jet",scale=2)
xlim(1.4689516128929263, 11)
ylim(38., 44.25205736596321)
colorbar(orientation="horizontal")
OceanPlot.plotmap()
OceanPlot.set_aspect_ratio()
title("surface current " * join(Dates.format.((minimum(timea),maximum(timea)),"yyyy-mm-dd")," - "))
savefig(expanduser("~/Figures/altimetry_currents_DIVAnd_zoom.png"),dpi=300)
=#

In [ ]:
#=
lon = [0,1]
lat = [50,50]
adt = [0, 1]

In [ ]:
perp_velocity(lon,lat,adt)

In [ ]:
lon = [0,1]
lat = [50,51]
adt = [0, 1]

In [ ]:
lonc,latc,u,v = perp_velocity(lon,lat,adt)
@test u[1] < 0
@test v[1] > 0

In [ ]:
lon = [1,0]
lat = [50,51]
adt = [0, 1]

In [ ]:
lonc,latc,u,v = perp_velocity(lon,lat,adt)
@test u[1] < 0
@test v[1] < 0

In [ ]:
lon = [1,0]
lat = [50,51]
adt = [1, 0]

In [ ]:
lonc,latc,u,v = perp_velocity(lon,lat,adt)
@test u[1] > 0
@test v[1] > 0

In [ ]:
@show perp_velocity(lon,lat,adt)
@show perp_velocity(reverse(lon),reverse(lat),reverse(adt))

In [ ]:
lon = [1,0]
lat = [-51,-50]
adt = [0, 1]

In [ ]:
lonc,latc,u,v = perp_velocity(lon,lat,adt)
@test u[1] > 0
@test v[1] > 0

In [ ]:
=#

=

In [ ]:
    * (i+1)



    * (i)

In [ ]:
=#

=

In [ ]:
(adt[i+1] - adt[i])
=#